In [11]:
from sqlalchemy import create_engine
import pandas as pd
import getpass
import time
import pickle
import os
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2
from nltk import word_tokenize
from nltk.corpus import stopwords
import nltk
import time
import logging
from scipy.sparse import hstack
def get_db_credentials():
    """Prompt user for MySQL credentials securely."""
    user = input("Enter MySQL username: ")
    password = getpass.getpass("Enter MySQL password: ")  # Hides password input
    return user, password

def fetch_data_from_databases(user, password, host, databases, cache_file="yelp_data_cache.pkl", force_refresh=False):
    """Fetch data from multiple databases with caching and detailed output."""
    if not force_refresh and os.path.exists(cache_file):
        print(f"\nLoading cached data from {cache_file}...")
        start_time = time.time()
        with open(cache_file, 'rb') as f:
            db_data = pickle.load(f)
        end_time = time.time()

        # Mimic the database fetch output for cached data
        for db in db_data:
            print(f"\nConnecting to {db} (cached)...")
            table_list = list(db_data[db].keys())
            print(f"Tables in {db}: {table_list}")
            for table in table_list:
                df = db_data[db][table]
                print(f"Fetching data from {table} (cached)...")
                print(f"Loaded {table} with {df.shape[0]} rows and {df.shape[1]} columns")
                print("Columns and Data Types:")
                for col, dtype in df.dtypes.items():
                    print(f"  {col}: {dtype}")
            print(f"Time taken for {db}: {end_time - start_time:.2f} seconds")

        print(f"\n✅ Cached data loaded! Total time: {end_time - start_time:.2f} seconds")
        return db_data

    # If no cache or force_refresh is True, fetch from databases
    db_data = {}
    total_start_time = time.time()

    for db in databases:
        print(f"\nConnecting to {db}...")
        start_time = time.time()

        # Create SQLAlchemy engine for the database
        engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{db}")

        # Get all table names from the database
        query_tables = "SHOW TABLES;"
        tables = pd.read_sql(query_tables, engine)
        table_list = tables.iloc[:, 0].tolist()
        print(f"Tables in {db}: {table_list}")

        # Store data for each table in a dictionary
        db_data[db] = {}

        # Loop through each table and load data
        for table in table_list:
            print(f"Fetching data from {table}...")
            query = f"SELECT * FROM {table};"
            df = pd.read_sql(query, engine)
            db_data[db][table] = df
            print(f"Loaded {table} with {df.shape[0]} rows and {df.shape[1]} columns")
            print("Columns and Data Types:")
            for col, dtype in df.dtypes.items():
                print(f"  {col}: {dtype}")

        end_time = time.time()
        print(f"Time taken for {db}: {end_time - start_time:.2f} seconds")

    total_end_time = time.time()
    print(f"\n✅ Data fetching complete! Total time: {total_end_time - total_start_time:.2f} seconds")

    # Save to cache
    print(f"Saving data to cache file: {cache_file}...")
    with open(cache_file, 'wb') as f:
        pickle.dump(db_data, f)
    print("✅ Cache saved successfully!")

    return db_data

In [12]:
host = "database-ml2025.cvk6uwuwc2j4.us-east-2.rds.amazonaws.com"
databases = ["yelp_hotel", "yelp_res"]
# Get credentials
user, password = get_db_credentials()
# Fetch data
data = fetch_data_from_databases(user, password, host, databases)

Enter MySQL username: admin
Enter MySQL password: ··········

Loading cached data from yelp_data_cache.pkl...

Connecting to yelp_hotel (cached)...
Tables in yelp_hotel: ['author_features', 'hotel', 'review', 'review_features', 'reviewer']
Fetching data from author_features (cached)...
Loaded author_features with 1716 rows and 7 columns
Columns and Data Types:
  Author_ID: object
  Author_ID_m: int64
  No_Of_Reviews: int64
  P_Filtered: float64
  MNR: int64
  BST: float64
  Review_Count: int64
Fetching data from hotel (cached)...
Loaded hotel with 282974 rows and 13 columns
Columns and Data Types:
  hotelID: object
  name: object
  location: object
  reviewCount: int64
  rating: int64
  categories: object
  address: object
  AcceptsCreditCards: object
  PriceRange: object
  WiFi: object
  webSite: object
  phoneNumber: object
  filReviewCount: float64
Fetching data from review (cached)...
Loaded review with 688313 rows and 10 columns
Columns and Data Types:
  date: object
  reviewID: o

In [15]:
# === Logging setup ===
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.FileHandler('training_with_ensemble.log'), logging.StreamHandler()]
)
logger = logging.getLogger(__name__)

# === Fast tokenizer without NLTK ===
stop_words = set([
    'the', 'and', 'is', 'in', 'to', 'it', 'of', 'for', 'on', 'this', 'that', 'with', 'as', 'was', 'but', 'are', 'have',
    'be', 'at', 'or', 'an', 'not', 'by', 'from', 'they', 'we', 'you', 'had', 'his', 'her', 'its', 'can', 'my', 'all',
    'if', 'there', 'been', 'so', 'no', 'out', 'up', 'what', 'when', 'which', 'who', 'would', 'will', 'just', 'about'
])

def extract_text_features(text):
    if pd.isna(text):
        return ''
    tokens = re.findall(r'\b\w+\b', text.lower())
    return ' '.join([word for word in tokens if word not in stop_words])

# === Data Preparation ===
def prepare_data(db_data, db_name, sample_frac=0.1):
    logger.info(f"Starting data preparation for {db_name}")
    start_time = time.time()

    reviews_df = db_data[db_name]['review'].sample(frac=sample_frac, random_state=42).copy()
    logger.info(f"Unique values in 'flagged' for {db_name}.review: {reviews_df['flagged'].unique().tolist()}")
    reviews_df['is_fake'] = reviews_df['flagged'].map({'YR': 1, 'Y': 1, 'NR': 0, 'N': 0})
    logger.info(f"Number of NaN in 'is_fake': {reviews_df['is_fake'].isna().sum()}")
    reviews_df = reviews_df.dropna(subset=['is_fake'])

    review_features_df = db_data[db_name]['review_features']
    author_features_df = db_data[db_name]['author_features']

    df = reviews_df.merge(review_features_df[['Review_ID', 'EXT', 'DEV', 'Filtered']],
                          left_on='reviewID', right_on='Review_ID', how='left')
    df = df.merge(author_features_df[['Author_ID', 'No_Of_Reviews', 'P_Filtered', 'MNR', 'BST']],
                  left_on='reviewerID', right_on='Author_ID', how='left')

    df['clean_text'] = df['reviewContent'].apply(extract_text_features)
    df['review_length'] = df['reviewContent'].str.len()

    logger.info(f"Finished data preparation for {db_name} in {time.time() - start_time:.2f} seconds")
    return df

# === Optional PCA (you can skip if memory is an issue) ===
def apply_pca(X, db_name):
    logger.info(f"Applying PCA for {db_name}")
    pca = PCA()
    X_pca = pca.fit_transform(X)
    explained_variance = pca.explained_variance_ratio_
    logger.info(f"PCA Explained Variance Ratios for {db_name}: {explained_variance[:10]}")
    logger.info(f"Cumulative Explained Variance for {db_name}: {np.cumsum(explained_variance)[:10]}")
    return X_pca, pca

# === Feature Selection (Chi-squared) ===
def select_features(X, y, db_name, k=50):
    logger.info(f"Selecting top {k} features for {db_name}")
    selector = SelectKBest(score_func=chi2, k=k)
    X_selected = selector.fit_transform(X, y)
    selected_indices = selector.get_support(indices=True)
    logger.info(f"Selected feature indices for {db_name}: {selected_indices}")
    return X_selected, selected_indices

# === Model Training and Evaluation ===
def train_and_evaluate(df, db_name):
    logger.info(f"Starting training for {db_name}")
    start_time = time.time()

    tfidf = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))
    X_text = tfidf.fit_transform(df['clean_text'])
    X_other = df[['rating', 'EXT', 'DEV', 'No_Of_Reviews', 'P_Filtered', 'MNR', 'BST', 'review_length']].fillna(0).values
    scaler = MinMaxScaler()
    X_other_scaled = scaler.fit_transform(X_other)

    # Use sparse stacking to avoid out-of-memory
    X = hstack([X_text, X_other_scaled])
    y = df['is_fake']

    # Optionally apply PCA (not recommended if running into memory issues)
    # X_pca, pca = apply_pca(X, db_name)

    # Feature Selection
    X_selected, selected_indices = select_features(X, y, db_name, k=50)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train_selected = X_train[:, selected_indices]
    X_test_selected = X_test[:, selected_indices]

    models = {
        'LinearSVC': LinearSVC(C=0.1, tol=1e-2, max_iter=1000),
        'Multinomial NB': MultinomialNB(),
        'Decision Tree': DecisionTreeClassifier(max_depth=10),
        'Logistic Regression': LogisticRegression(max_iter=1000),
        'SGD': SGDClassifier(max_iter=1000, tol=1e-3),
        'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42),
        'XGBoost': XGBClassifier(n_estimators=100, max_depth=10, random_state=42, eval_metric='logloss'),
        'Stacking': StackingClassifier(
            estimators=[
                ('lr', LogisticRegression(max_iter=1000)),
                ('xgb', XGBClassifier(n_estimators=100, max_depth=5, random_state=42, eval_metric='logloss')),
                ('rf', RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42))
            ],
            final_estimator=LogisticRegression(max_iter=1000),
            cv=5,
            n_jobs=1
        )
    }

    results = {}
    for name, model in models.items():
        logger.info(f"Training {name} for {db_name}")
        model_start = time.time()

        if name == 'Stacking':
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
        elif name in ['Random Forest', 'XGBoost']:
            model.fit(X_train_selected, y_train)
            y_pred = model.predict(X_test_selected)
        else:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

        model_end = time.time()

        results[name] = {
            'Precision': precision_score(y_test, y_pred),
            'Recall': recall_score(y_test, y_pred),
            'F1': f1_score(y_test, y_pred),
            'Accuracy': accuracy_score(y_test, y_pred),
            'Time (s)': model_end - model_start
        }
        logger.info(f"Finished training {name} for {db_name} in {model_end - model_start:.2f} seconds")

    logger.info(f"Completed training for {db_name} in {time.time() - start_time:.2f} seconds")
    return results

In [16]:
# === Main Driver ===
start_total = time.time()
logger.info("Starting entire process")

# Load your cached data
hotel_df = prepare_data(data, 'yelp_hotel', sample_frac=1)
rest_df = prepare_data(data, 'yelp_res', sample_frac=1)

hotel_results = train_and_evaluate(hotel_df, 'yelp_hotel')
rest_results = train_and_evaluate(rest_df, 'yelp_res')

# === Summarize and PRINT OUTPUT ===
logger.info("=== Model Performance Summary ===")

for db_name, results in [('yelp_hotel', hotel_results), ('yelp_res', rest_results)]:
    print(f"\n=== Results for {db_name} ===")
    df_results = pd.DataFrame(results).T.round(3)
    print(df_results)
    print("\n")

total_time = time.time() - start_total
logger.info(f"Total runtime: {total_time:.2f} seconds")
print(f"Total runtime: {total_time:.2f} seconds")


=== Results for yelp_hotel ===
                     Precision  Recall     F1  Accuracy  Time (s)
LinearSVC                0.637   0.503  0.562     0.695     7.409
Multinomial NB           0.609   0.429  0.504     0.670     0.232
Decision Tree            0.577   0.371  0.452     0.649    76.377
Logistic Regression      0.635   0.517  0.570     0.696     4.261
SGD                      0.646   0.455  0.534     0.690     2.117
Random Forest            0.604   0.254  0.358     0.644    97.504
XGBoost                  0.593   0.432  0.500     0.663    15.661
Stacking                 0.636   0.538  0.583     0.700  1211.116



=== Results for yelp_res ===
                     Precision  Recall     F1  Accuracy  Time (s)
LinearSVC                0.594   0.457  0.516     0.644     4.657
Multinomial NB           0.570   0.361  0.443     0.621     0.258
Decision Tree            0.580   0.216  0.314     0.609    90.247
Logistic Regression      0.594   0.470  0.525     0.646     7.371
SGD         